In [1]:
from utils import *
import numpy as np
import torch, os
from sklearn.preprocessing import StandardScaler
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import GridSearchCV

c:\Python310\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
set_random_seed(3)

Seed: 3


In [3]:
# def scaling_embeddings(embeddings):
#     for i in range(len(embeddings)):
#         embeddings[i] = StandardScaler().fit_transform(embeddings[i])
#     return embeddings

def reshape_embeddings(embeddings):
    if embeddings.shape[1] == 1:
        embeddings = embeddings.squeeze(1)
    return embeddings

In [4]:
dynahate_labels_train = process_labels(read_labels("dynahate", "train"))
dynahate_labels_dev = process_labels(read_labels("dynahate", "dev"))
dynahate_labels_test = process_labels(read_labels("dynahate", "test"))

latenthatred_labels_train = read_labels("latenthatred", "train")
latenthatred_labels_dev = read_labels("latenthatred", "dev")
latenthatred_labels_test = read_labels("latenthatred", "test")

olid_labels_train = read_labels("olid", "train")
olid_labels_dev = read_labels("olid", "dev")
olid_labels_test = read_labels("olid", "test")

In [5]:
mlp = MLPClassifier(random_state=3)
gridsearch = GridSearchCV(
    mlp,
    param_grid={
        "hidden_layer_sizes": [(128), (128,64)],
        "activation": ["relu"],
        "solver": ["adam"],
        "learning_rate_init": [0.001, 0.0001],
        "learning_rate": ["adaptive"],
        "early_stopping": [True],
        "max_iter": [10000]
    },
    verbose=4,
    n_jobs=os.cpu_count()//3,
    cv=5,
)

In [6]:
dynahate_labels_train_dev = np.concatenate((dynahate_labels_train, dynahate_labels_dev))
latenthatred_labels_train_dev = np.concatenate((latenthatred_labels_train, latenthatred_labels_dev))
olid_labels_train_dev = np.concatenate((olid_labels_train, olid_labels_dev))

In [7]:
hatebert_dynahate_train_embeddings = reshape_embeddings(hatebert_dynahate_train_embeddings)
hatebert_dynahate_dev_embeddings = reshape_embeddings(hatebert_dynahate_dev_embeddings)
hatebert_dynahate_test_embeddings = reshape_embeddings(hatebert_dynahate_test_embeddings)

hatebert_latenthatred_train_embeddings = reshape_embeddings(hatebert_latenthatred_train_embeddings)
hatebert_latenthatred_dev_embeddings = reshape_embeddings(hatebert_latenthatred_dev_embeddings)
hatebert_latenthatred_test_embeddings = reshape_embeddings(hatebert_latenthatred_test_embeddings)

hatebert_olid_train_embeddings = reshape_embeddings(hatebert_olid_train_embeddings)
hatebert_olid_dev_embeddings = reshape_embeddings(hatebert_olid_dev_embeddings)
hatebert_olid_test_embeddings = reshape_embeddings(hatebert_olid_test_embeddings)

### HateBERT

#### DynaHate

In [8]:
hatebert_dynahate_train_dev = np.concatenate((hatebert_dynahate_train_embeddings, hatebert_dynahate_dev_embeddings))

grid_results = gridsearch.fit(hatebert_dynahate_train_dev, dynahate_labels_train_dev)

best_params = grid_results.best_params_
mlp = grid_results.best_estimator_

print("Best params: ", best_params)

train_dev_preds = mlp.predict(hatebert_dynahate_train_dev)
test_preds = mlp.predict(hatebert_dynahate_test_embeddings)

computeAllScores(train_dev_preds, test_preds, dynahate_labels_train_dev, dynahate_labels_test, "Results/hatebert_dynahate_seed_3")
save_model(mlp, "Saves/hatebert_dynahate_seed_3")

Fitting 5 folds for each of 4 candidates, totalling 20 fits
Best params:  {'activation': 'relu', 'early_stopping': True, 'hidden_layer_sizes': (128, 64), 'learning_rate': 'adaptive', 'learning_rate_init': 0.001, 'max_iter': 10000, 'solver': 'adam'}
Accuracy Train Dev:  0.7227744165946414
Accuracy Test:  0.6844660194174758
Weighted F1 Train Dev:  0.7217386198208466
Weighted F1 Test:  0.6829655935355753
Macro F1 Train Dev:  0.7194304145501702
Macro F1 Test:  0.6785699796843745
Micro F1 Train Dev:  0.7227744165946414
Micro F1 Test:  0.6844660194174758
Weighted Recall Train Dev:  0.7227744165946414
Weighted Recall Test:  0.6844660194174758
Macro Recall Train Dev:  0.7186283690094266
Macro Recall Test:  0.677700069708709
Micro Recall Train Dev:  0.7227744165946414
Micro Recall Test:  0.6844660194174758
Confusion Matrix Train Dev: 
[[11359  5758]
 [ 4506 15401]]
Confusion Matrix Test: 
[[1131  721]
 [ 579 1689]]


#### LatentHatred

In [9]:
hatebert_latenthatred_train_dev = np.concatenate((hatebert_latenthatred_train_embeddings, hatebert_latenthatred_dev_embeddings))

grid_results = gridsearch.fit(hatebert_latenthatred_train_dev, latenthatred_labels_train_dev)

best_params = grid_results.best_params_
mlp = grid_results.best_estimator_

print("Best params: ", best_params)

train_dev_preds = mlp.predict(hatebert_latenthatred_train_dev)
test_preds = mlp.predict(hatebert_latenthatred_test_embeddings)

computeAllScores(train_dev_preds, test_preds, latenthatred_labels_train_dev, latenthatred_labels_test, "Results/hatebert_latenthatred_seed_3")
save_model(mlp, "Saves/hatebert_latenthatred_seed_3")

Fitting 5 folds for each of 4 candidates, totalling 20 fits
Best params:  {'activation': 'relu', 'early_stopping': True, 'hidden_layer_sizes': (128, 64), 'learning_rate': 'adaptive', 'learning_rate_init': 0.001, 'max_iter': 10000, 'solver': 'adam'}
Accuracy Train Dev:  0.703227808814401
Accuracy Test:  0.6849162011173184
Weighted F1 Train Dev:  0.6769652743294887
Weighted F1 Test:  0.6558743124989679
Macro F1 Train Dev:  0.4891614648056841
Macro F1 Test:  0.4633432913657562
Micro F1 Train Dev:  0.703227808814401
Micro F1 Test:  0.6849162011173184
Weighted Recall Train Dev:  0.703227808814401
Weighted Recall Test:  0.6849162011173184
Macro Recall Train Dev:  0.4747892239647013
Macro Recall Test:  0.4524785808161331
Micro Recall Train Dev:  0.703227808814401
Micro Recall Test:  0.6849162011173184
Confusion Matrix Train Dev: 
[[8821 1094   30]
 [2860 2442   45]
 [ 401  351   66]]
Confusion Matrix Test: 
[[2934  398   14]
 [1016  726   11]
 [ 151  102   18]]


#### OLID

In [10]:
hatebert_olid_train_dev = np.concatenate((hatebert_olid_train_embeddings, hatebert_olid_dev_embeddings))

grid_results = gridsearch.fit(hatebert_olid_train_dev, olid_labels_train_dev)

best_params = grid_results.best_params_
mlp = grid_results.best_estimator_

print("Best params: ", best_params)

train_dev_preds = mlp.predict(hatebert_olid_train_dev)
test_preds = mlp.predict(hatebert_olid_test_embeddings)

computeAllScores(train_dev_preds, test_preds, olid_labels_train_dev, olid_labels_test, "Results/hatebert_olid_seed_3")

save_model(mlp, "Saves/hatebert_olid_seed_3")

Fitting 5 folds for each of 4 candidates, totalling 20 fits
Best params:  {'activation': 'relu', 'early_stopping': True, 'hidden_layer_sizes': 128, 'learning_rate': 'adaptive', 'learning_rate_init': 0.001, 'max_iter': 10000, 'solver': 'adam'}
Accuracy Train Dev:  0.7492447129909365
Accuracy Test:  0.7872093023255814
Weighted F1 Train Dev:  0.7376992182423182
Weighted F1 Test:  0.7657245612518372
Macro F1 Train Dev:  0.6941830920251142
Macro F1 Test:  0.6879702602230484
Micro F1 Train Dev:  0.7492447129909365
Micro F1 Test:  0.7872093023255814
Weighted Recall Train Dev:  0.7492447129909365
Weighted Recall Test:  0.7872093023255814
Macro Recall Train Dev:  0.683854895104895
Macro Recall Test:  0.6685483870967742
Micro Recall Train Dev:  0.7492447129909365
Micro Recall Test:  0.7872093023255814
Confusion Matrix Train Dev: 
[[7769 1071]
 [2249 2151]]
Confusion Matrix Test: 
[[581  39]
 [144  96]]
